### Importing required libraries

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import keras

### Reading the data

In [ ]:
(train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.cifar10.load_data()
train_imgs = train_imgs/ 255.0
test_imgs = test_imgs/ 255.0

num_classes = 10
train_lbls = np.squeeze(keras.utils.to_categorical(train_lbls, num_classes))
test_lbls = np.squeeze(keras.utils.to_categorical(test_lbls, num_classes))

### Dispaly of data

In [ ]:
img = (train_imgs[400].squeeze())
plt.imshow(img, cmap='gray')

### Creating model 'B'

In [ ]:
from tensorflow.keras import backend
from tensorflow.keras.layers import Layer
class LocalResponseNormalization(Layer):

    def __init__(self, n=5, alpha=0.0001, beta=0.5, c=1, **kwargs):

        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.c = c
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x, mask=None):
        local_response = tf.nn.local_response_normalization(x, depth_radius = self.n, bias = self.c, alpha = self.alpha, beta = self.beta)
        return local_response

In [ ]:
class BasicLayers(object):

    def __init__(self, filters, kernel_size):

        self.b_conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', use_bias=False)
        self.l_conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', use_bias=False)
        self.t_conv = tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides=(2,2), padding='same', use_bias=False)

    def __call__(self, b_input=None, l_input=None, t_input=None):
        conv_list = []

        if b_input is not None:
            conv_list.append(self.b_conv(b_input))

        if l_input is not None:
            conv_list.append(self.l_conv(l_input))
          
        if t_input is not None:
            conv_list.append(self.t_conv(t_input))

        return tf.add_n(conv_list)


In [ ]:
def blt_net(input_tensor, classes, time_steps):

        layers = [BasicLayers(32, 3),
            BasicLayers(32, 3)]

        n_layers = len(layers)
        hidden_states = [[None for _ in range(n_layers)]
                       for _ in range(time_steps)]
        after_globalmax = [None for _ in range(time_steps)]
        outputs = [None for _ in range(time_steps)]

        for t in range(time_steps):
            for n, layer in enumerate(layers):

                if n == 0:
                    b_input = input_tensor 
                else:
                    b_input = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(hidden_states[t][n-1])


                if t == 0:
                    l_input = None
                else:
                    l_input = hidden_states[t-1][n]
                

                if t!=0 and n==0:
                    t_input = hidden_states[t-1][n+1]
                    print((t_input.shape))
                else:
                    t_input = None

                x = layer(b_input, l_input, t_input)
                relu_output = tf.keras.layers.Activation('relu')(x)
                hidden_states[t][n] = LocalResponseNormalization()(relu_output)

            x = tf.keras.layers.GlobalMaxPool2D()(hidden_states[t][-1])
            after_globalmax[t] = tf.keras.layers.Dense(classes, kernel_regularizer=tf.keras.regularizers.l2(0.0005))(x)

            if t > 0:
                x = tf.add_n(after_globalmax[:t+1])
            else:
                x = after_globalmax[t]

            outputs[t] = tf.keras.layers.Activation('sigmoid')(x)

        model = tf.keras.Model(inputs=input_tensor,outputs=outputs)    

        return model

In [ ]:
inputs = tf.keras.layers.Input(shape = (32,32,3))
model = blt_net(inputs, 10, 4)
model.summary()

### Initializing parameters for training

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import callbacks

def step_decay(epoch):
    initial_lr=0.001
    decay_rate = 0.01
    decay_step = 30
    lrate = initial_lr * decay_rate ** (epoch/decay_step)
    print("learning_rate")
    print(lrate)
    return lrate

Adam = tf.keras.optimizers.Adam(lr=0.0)
model.compile(loss='categorical_crossentropy', optimizer = Adam, metrics=['accuracy'])

lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

### Compiling the model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = Adam, metrics=['accuracy'])

### Fit the model

In [ ]:
# This is used with the dataset where there is no noise.
history = model.fit([train_imgs, train_imgs, train_imgs, train_imgs], [train_lbls, train_lbls, train_lbls, train_lbls], epochs=100, batch_size=128, validation_split=0.1, callbacks = callbacks_list)

In [ ]:
print(history.history.keys())

### Display learning curve

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Test the model

In [ ]:
#without Noise
model.evaluate([test_imgs, test_imgs, test_imgs, test_imgs], [test_lbls,test_lbls,test_lbls,test_lbls])

### Predictions of the model

In [ ]:
predictions = model.predict(test_imgs)

In [ ]:
n = 4
plt.figure(figsize=(15,15))
for i in range(n):
    plt.subplot(1,n,i+1)
    plt.imshow(test_imgs[i])
    plt.title("Lable:{}\nPredicted:{}".format(test_lbls[i],np.argmax(predictions[i])))
    plt.axis='off'
plt.show()

### Saving the model and the predictions

In [ ]:
model.save_weights('Model_BLT.h5') #Depending on the level of noise, name is given

In [ ]:
model_predictions = []
ground_truth_values = []
for i in range(len(test_imgs)):
    model_predictions.append(np.argmax(predictions[i]))
      ground_truth_values.append(np.argmax(test_lbls[i]))


correct_prediction = [i if i==j else 0 for i, j in zip(model_predictions, ground_truth_values)]
import pandas as pd
prediction_df = pd.DataFrame(correct_prediction, columns=['result']).to_csv('BLT_1_prediction.csv', index=False)